In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.FIM import FIMModel

In [2]:
hparams = {
    'scale':'demo',
    'name':'fim',
    'batch_size':5,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'group_auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:0',
    'attrs': ['title'],
    'epochs':5,
    'k':-1,
    'save_step':None,
    'save_each_epoch':True,
    'train_embedding':False,
}
device = torch.device(hparams['device'])

In [3]:
vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)

In [4]:
fimModel = FIMModel(vocab=vocab,hparams=hparams).to(device)

In [5]:
train(fimModel, hparams, loader_train, loader_test, loader_validate)

training...
epoch 1 , step 300 , loss: 1.5360: : 400it [00:29, 13.59it/s]


KeyError: 'select'